In [ ]:
#xX3rCbSDM4vF0QEfgh09b2ZWW

## Data Preprocessing

#### Downloading data from NYC Open Data

In [2]:
import pandas as pd
from sodapy import Socrata


In [1]:
#small sample trail
query="""
select * 
where
    created_date between '2015-01-01T00:00:00.000' 
    and '2015-12-31T23:59:59.999'
"""
#'2015-01-01T00:00:00.000'

In [5]:

import time
from requests.exceptions import ReadTimeout
from sodapy import Socrata

# Example authenticated client (needed for non-public datasets):
client = Socrata( "data.cityofnewyork.us",
                  "xX3rCbSDM4vF0QEfgh09b2ZWW",
                  username="yirong263@gmail.com",
                  password="UTDYnmz*zn2u3g6",
                  timeout=60)
# Variables for retry logic
max_retries = 5
retry_wait = 10  # Initial wait time in seconds

while max_retries > 0:
    try:    
    # Set initial parameters for the SoQL query
        limit = 1000000  # Example limit
        offset = 0  # Start at the beginning
        total_records = 100000000  # Example total number of records you wish to download
        current_record = 0
        while current_record < total_records:
            # Adjust the query to include the limit and offset
            results = client.get("erm2-nwe9",query=query+ f" limit {limit} offset {offset}")
            
            # Convert to DataFrame and save to CSV
            df = pd.DataFrame.from_records(results)
            df.to_csv(f'nyc_311_data_part_2015_12_31_{offset // limit + 1}.csv', index=False)
            
            # Update the offset and current_record count
            offset += limit
            current_record += len(results)

            # Optional: Print progress
            print(f'Downloaded {current_record} of {total_records}')
        break
    
    except ReadTimeout:
        # Wait before retrying
        time.sleep(retry_wait)
        # Reduce the number of retries left
        max_retries -= 1
        # Increase the wait time for the next retry
        retry_wait *= 2


#results = client.get("erm2-nwe9",query=query)
# Convert to pandas DataFrame
#NYC_311df = pd.DataFrame.from_records(results)
#print(NYC_311df)

In [1]:
#condense all of the files

import pandas as pd
import glob
import re
# After downloading all chunks

csv_files = glob.glob('nyc_311_data_*.csv')

# Remove unnecessary columns by keeping only the ones you need for each file
dfs=[]
for file in csv_files:
   df=pd.read_csv(file)
   columns_needed = ['unique_key', 'created_date', 'complaint_type','incident_zip','latitude', 'longitude']  # Replace with actual column names
   df = df[columns_needed]
   #eliminate duplicate
   df.drop_duplicates(inplace=True)
   # Remove invalid data points
   # This is highly dependent on the context of your data, but as an example:
   df.dropna(inplace=True)  # Drop rows where 'column1' or 'column2' is NaN
   # Normalize column names
   df.columns = [column_name.lower().replace(' ', '_') for column_name in df.columns]
   
   dfs.append(df) # processed df and append to a list

# Concatenate all DataFrames into one
NYC311_df = pd.concat(dfs,ignore_index=True)
NYC311_df.drop_duplicates(inplace=True)# header duplicate elimination
# Optionally, save the combined DataFrame to a new CSV file
NYC311_df.to_csv('nyc_311_data.csv', index=False)

/var/folders/3q/9w4rbrs12tv6_t2_bscp34qm0000gn/T/ipykernel_11273/1055486703.py:13: DtypeWarning: Columns (8,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file)
/var/folders/3q/9w4rbrs12tv6_t2_bscp34qm0000gn/T/ipykernel_11273/1055486703.py:13: DtypeWarning: Columns (8,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file)
/var/folders/3q/9w4rbrs12tv6_t2_bscp34qm0000gn/T/ipykernel_11273/1055486703.py:13: DtypeWarning: Columns (8,33,34,35,36,37,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file)
/var/folders/3q/9w4rbrs12tv6_t2_bscp34qm0000gn/T/ipykernel_11273/1055486703.py:13: DtypeWarning: Columns (8,34,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file)
/var/folders/3q/9w4rbrs12tv6_t2_bscp34qm0000gn/T/ipykernel_11273/1055486703.py:13: DtypeWarning: Columns (38,39,40) have mixed types. 

In [1]:
import pandas as pd
NYC311_df= pd.read_csv('/Users/jadeli/Documents/GitHub/IEOR4501-Project/nyc_311_data.csv')
NYC311_df.head(5)

,unique_key,created_date,complaint_type,incident_zip,latitude,longitude
0,29991956,2015-02-19T00:00:00.000,HEAT/HOT WATER,11207.0,40.687162,-73.912622
1,29991952,2015-02-19T00:00:00.000,HEAT/HOT WATER,10030.0,40.813442,-73.943315
2,29991943,2015-02-19T00:00:00.000,HEAT/HOT WATER,10468.0,40.865775,-73.901848
3,29991924,2015-02-19T00:00:00.000,GENERAL,10029.0,40.789431,-73.943295
4,29991914,2015-02-19T00:00:00.000,GENERAL,10032.0,40.841254,-73.937794


In [2]:
import geopandas as gpd
# Normalize Column Types

# unique_key 
NYC311_df['unique_key'] = NYC311_df['unique_key'].astype(int)
# change name into 'id_NYC311'
NYC311_df.rename(columns={'unique_key': 'id_NYC311'}, inplace=True)

#incident zip
#rename from incident_zip to zipcode
NYC311_df.rename(columns={'incident_zip': 'zipcode'}, inplace=True)
# Convert the 'zipcode' column to a string type, then filter
NYC311_df = NYC311_df[NYC311_df['zipcode'].astype(str).str.len() == 5]
NYC311_df['zipcode']=NYC311_df['zipcode'].astype(int)

#created_date
#rename "date"
NYC311_df.rename(columns={'created_date': 'date'}, inplace=True)
#change date format into yyyy-mm-dd
NYC311_df['date'] = pd.to_datetime(NYC311_df['date']).dt.strftime('%Y-%m-%d')
    
# Assuming df is your existing DataFrame with latitude and longitude columns
gdf = gpd.GeoDataFrame(NYC311_df, geometry=gpd.points_from_xy(NYC311_df['longitude'], NYC311_df['latitude']))
gdf.crs = "EPSG:4326"  # Set the original CRS to WGS84
target_srid = "EPSG:3857"  # Define the target CRS (Web Mercator)
gdf = gdf.to_crs(target_srid)  # Transform the CRS to the target


In [23]:
from datetime import datetime
start_time=datetime(2015, 1, 1)
end_time=datetime(2015, 12, 31)
query=f"""
select * 
where
    created_at between '{start_time.strftime('%m/%d/%Y')}' 
    and '{end_time.strftime('%m/%d/%Y')}'
"""
print(query)


select * 
where
    created_at between '01/01/2015' 
    and '12/31/2015'



In [ ]:
#2015-1-1~2015-12-31
import time
from requests.exceptions import ReadTimeout
from sodapy import Socrata

# Example authenticated client (needed for non-public datasets):
client = Socrata( "data.cityofnewyork.us",
                  "xX3rCbSDM4vF0QEfgh09b2ZWW",
                  username="yirong263@gmail.com",
                  password="UTDYnmz*zn2u3g6",
                  timeout=60)
# Variables for retry logic
max_retries = 5
retry_wait = 10  # Initial wait time in seconds
while max_retries > 0:
    try:    
    # Set initial parameters for the SoQL query
        limit = 200000  # Example limit
        offset = 0  # Start at the beginning
        total_records = 10000  # Example total number of records you wish to download
        current_record = 0
        while current_record < total_records:
            # Adjust the query to include the limit and offset
            results = client.get("5rq2-4hqu",query=query+ f"limit {limit} offset {offset}")
            
            # Convert to DataFrame and save to CSV
            df = pd.DataFrame.from_records(results)
            df.to_csv(f'tree_{offset // limit + 1}.csv', index=False)
            
            # Update the offset and current_record count
            offset += limit
            current_record += len(results)

            # Optional: Print progress
            print(f'Downloaded {current_record} of {total_records}')
        break
    
    except ReadTimeout:
        # Wait before retrying
        time.sleep(retry_wait)
        # Reduce the number of retries left
        max_retries -= 1
        # Increase the wait time for the next retry
        retry_wait *= 2

# Convert to pandas DataFrame
tree_df = pd.DataFrame.from_records(results)

In [23]:
import pandas as pd
tree_df= pd.read_csv('/Users/jadeli/Documents/GitHub/IEOR4501-Project/tree.csv')
tree_df.head(5)
# Convert the column names to a list and print them
column_names = tree_df.columns.tolist()
print(column_names)

['created_at', 'tree_id', 'block_id', 'the_geom', 'tree_dbh', 'stump_diam', 'curb_loc', 'status', 'health', 'spc_latin', 'spc_common', 'steward', 'guards', 'sidewalk', 'user_type', 'problems', 'root_stone', 'root_grate', 'root_other', 'trnk_wire', 'trnk_light', 'trnk_other', 'brnch_ligh', 'brnch_shoe', 'brnch_othe', 'address', 'zipcode', 'zip_city', 'cb_num', 'borocode', 'boroname', 'cncldist', 'st_assem', 'st_senate', 'nta', 'nta_name', 'boro_ct', 'state', 'latitude', 'longitude', 'x_sp', 'y_sp']


In [25]:
# Remove unnecessary columns by keeping only the ones you need
columns_needed = ['created_at', 'tree_id', 'status','zipcode','health','spc_common', 'latitude', 'longitude']  # Replace with actual column names
tree_df = tree_df[columns_needed]

# Remove invalid data points
# This is highly dependent on the context of your data, but as an example:
tree_df.drop_duplicates(inplace=True)
tree_df.dropna(inplace=True)  

# Normalize column names
tree_df.columns = [column_name.lower().replace(' ', '_') for column_name in tree_df.columns]
#created_at
tree_df.rename(columns={'created_at': 'date'}, inplace=True)
tree_df['date'] = pd.to_datetime(tree_df['date']).dt.strftime('%Y-%m-%d')#change date format into yyyy-mm-dd

#zipcode
tree_df['zipcode'] = tree_df['zipcode'].astype(int)
